In [1]:
f = open("PRDCC_000001.cif","r")
lines = f.readlines()
f.close()

In [2]:
print(lines)

['data_PRDCC_000001\n', '# \n', '_chem_comp.id                                    PRD_000001 \n', '_chem_comp.name                                  "Actinomycin D" \n', '_chem_comp.formula                               "C62 H86 N12 O16" \n', '_chem_comp.formula_weight                        1255.417 \n', '_chem_comp.pdbx_release_status                   REF_ONLY \n', '_chem_comp.type                                  peptide-like \n', '_chem_comp.pdbx_type                             ? \n', '_chem_comp.mon_nstd_parent_comp_id               ? \n', '_chem_comp.pdbx_synonyms                         ? \n', '_chem_comp.pdbx_formal_charge                    ? \n', '_chem_comp.pdbx_initial_date                     ? \n', '_chem_comp.pdbx_modified_date                    ? \n', '_chem_comp.pdbx_ambiguous_flag                   ? \n', '_chem_comp.pdbx_replaced_by                      ? \n', '_chem_comp.pdbx_replaces                         ? \n', '_chem_comp.one_letter_code                      

In [3]:
block=[]
for i,l in enumerate(lines):
    if 'loop' in l:
        print(i)
        block.append(i)
print (block)    

print(lines[block[1]+1])

27
227
418
432
[27, 227, 418, 432]
_chem_comp_bond.comp_id 



In [4]:
chem_comp_atom_header=[]
for i in range(block[0]+1,block[1]):
    if '_chem_comp_atom' in lines[i]:
        chem_comp_atom_header.append(lines[i].replace('_chem_comp_atom.','').replace(' \n',''))
    else:
        dataframe_nodes_start=i
        break

print(*chem_comp_atom_header, sep = "\n")
print(dataframe_nodes_start)

comp_id
atom_id
alt_atom_id
type_symbol
charge
pdbx_align
pdbx_aromatic_flag
pdbx_leaving_atom_flag
pdbx_stereo_config
model_Cartn_x
model_Cartn_y
model_Cartn_z
pdbx_model_Cartn_x_ideal
pdbx_model_Cartn_y_ideal
pdbx_model_Cartn_z_ideal
pdbx_component_comp_id
pdbx_residue_numbering
pdbx_component_atom_id
pdbx_polymer_type
pdbx_ref_id
pdbx_component_id
pdbx_ordinal
50


In [5]:
chem_comp_bond_header=[]
for i in range(block[1]+1,block[2]):
    if '_chem_comp_bond' in lines[i]:
        chem_comp_bond_header.append(lines[i].replace('_chem_comp_bond.','').replace(' \n',''))
    else:
        dataframe_edges_start=i
        break

print(*chem_comp_bond_header, sep = "\n")
print(dataframe_edges_start)

comp_id
atom_id_1
atom_id_2
value_order
pdbx_aromatic_flag
pdbx_stereo_config
pdbx_ordinal
235


In [6]:
data=[]
for i in range (dataframe_nodes_start,block[1]-1):
    data.append(lines[i].split())

In [7]:
import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkContext
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext
from pyspark.sql.functions import *

In [ ]:
df_nodes = spark.createDataFrame(data=data, schema = chem_comp_atom_header)
df_nodes.select('atom_id','type_symbol','model_Cartn_x','model_Cartn_y','model_Cartn_z').show(5)

In [ ]:
data=[]
for i in range (dataframe_edges_start,block[2]-1):
    data.append(lines[i].split())

In [ ]:
df_edges = spark.createDataFrame(data=data, schema = chem_comp_bond_header)
df_edges.show(5)

In [ ]:
from __future__  import print_function  # Python 2/3 compatibility

from gremlin_python import statics
from gremlin_python.structure.graph import Graph
from gremlin_python.process.graph_traversal import __, GraphTraversalSource
from gremlin_python.process.strategies import *
from gremlin_python.driver.driver_remote_connection import DriverRemoteConnection

In [ ]:
graph = Graph()

remoteConn = DriverRemoteConnection('ws://localhost:8182/gremlin','g')
g = graph.traversal().withRemote(remoteConn)

In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
g.V().count().next()  # to validate connection, should return ‘0’

In [ ]:
df_nodes_collection = df_nodes.collect()
for row in df_nodes_collection:
    g.addV(row['type_symbol']).property('atom_id', row['atom_id']).next()
g.V().count().next()

In [ ]:
df_edges.show(3)

In [ ]:
df_edges_collection = df_edges.collect()

for row in df_edges_collection:
    g.V(g.V().has('atom_id',row['atom_id_1']).next())\
    .addE(row['value_order'])\
    .to(__.V(g.V().has('atom_id',row['atom_id_2']).next())).next()

In [ ]:
connections = g.E().valueMap(True).toList()
print(connections)